# สร้างแบบจำลอง Random Forest และใช้ปรับแก้ฝนเรดาร์รายชั่วโมง
* ผลของการทดสอบวิธีการปรับแก้ผลิตภัณฑ์ฝนโมเสคลุ่มเจ้าพระยา ด้วยวิธีการหลากหลาย พบว่าการใช้ Random Forest ที่เป็นหนึ่งในแมชชีนเลิร์นนิ่ง ให้ผลการปรับแก้ที่ดีเมื่อเทียบกับจำนวนสถานีฝนภาคพื้นดิน (validating stations)
* ดังนั้นในโค้ดนี้จึงสร้างแบบจำลอง RF โดยการรวมฝนเรดาร์ให้เป็นฝนรายเหตุการณ์ก่อน แล้ว train/test ด้วยฝนสถานีที่รวมเป็นฝนรายเหตุการณ์เช่นกัน ด้วยวิธีการ grid search เพื่อให้ได้ best parameters แล้วจึงเซฟโมเดลออกไป เพื่อเรียกใช้ในการปรับแก้ฝนรายชั่วโมงต่อไป

## สร้างแบบจำลอง Random Forest using Grid search แล้วเซฟเพื่อนำไปใช้ปรับแก้ฝนเรดาร์

In [9]:
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import rasterio
import logging
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import joblib

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Function to calculate evaluation metrics
def calculate_metrics(observed, predicted):
    metrics = {}
    metrics['MBE'] = np.mean(predicted - observed)
    metrics['MAE'] = np.mean(np.abs(predicted - observed))
    metrics['RMSE'] = np.sqrt(np.mean((predicted - observed)**2))
    metrics['PBIAS'] = 100 * np.sum(predicted - observed) / np.sum(observed) if np.sum(observed) != 0 else np.nan
    metrics['NSE'] = 1 - (np.sum((observed - predicted)**2) / np.sum((observed - np.mean(observed))**2)) if np.sum((observed - np.mean(observed))**2) != 0 else np.nan
    metrics['R'] = np.corrcoef(observed, predicted)[0, 1] if np.std(observed) != 0 and np.std(predicted) != 0 else np.nan
    return metrics

def calculate_total_radar_rainfall(radar_dir):
    radar_files = glob(os.path.join(radar_dir, '*.tif'))
    total_rainfall = None
    transform = None
    
    for file in tqdm(radar_files, desc="Processing radar files"):
        with rasterio.open(file) as src:
            radar_data = src.read(1)  # Read the first band
            if total_rainfall is None:
                total_rainfall = radar_data
                transform = src.transform
            else:
                total_rainfall += radar_data
    
    return total_rainfall, transform

def extract_radar_rainfall(lon, lat, total_rainfall, transform, window_size=3):
    row, col = rasterio.transform.rowcol(transform, lon, lat)
    half_window = window_size // 2
    window = total_rainfall[max(0, row-half_window):row+half_window+1, 
                            max(0, col-half_window):col+half_window+1]
    return np.nanmean(window)

# Function to calculate MFB
def calculate_mfb(observed, predicted):
    return np.sum(observed) / np.sum(predicted)

def optimize_random_forest(train_df, test_df):
    features = ['radar_rainfall', 'longitude', 'latitude']
    X_train = train_df[features]
    y_train = train_df['rain']
    X_test = test_df[features]
    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                               cv=20, n_jobs=-1, verbose=2, scoring='r2') #cv=20
    
    grid_search.fit(X_train, y_train)
    
    logging.info(f"Best parameters found: {grid_search.best_params_}")
    logging.info(f"Best cross-validation score: {grid_search.best_score_}")
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    feature_importance = pd.DataFrame({'feature': features, 'importance': best_model.feature_importances_})
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    logging.info("Feature Importance:")
    logging.info(feature_importance)
    
    return y_pred, best_model

def process_fold(fold):    
    logging.info(f"Processing fold {fold}")
    
    train_df = pd.read_csv(f'./train_test_validate/train_split_{fold}.csv')
    test_df = pd.read_csv(f'./train_test_validate/test_split_{fold}.csv')
    
    # Original (no correction)
    mfb_train = calculate_mfb(train_df['rain'], train_df['radar_rainfall'])
    mfb_test = calculate_mfb(test_df['rain'], test_df['radar_rainfall'])
    metrics_original = calculate_metrics(test_df['rain'], test_df['radar_rainfall'])
    
    # Optimized Random Forest bias correction
    test_df['radar_rainfall_rf'], best_model = optimize_random_forest(train_df, test_df)
    mfb_test_rf = calculate_mfb(test_df['rain'], test_df['radar_rainfall_rf'])
    metrics_rf = calculate_metrics(test_df['rain'], test_df['radar_rainfall_rf'])
    
    # Calculate R2 score for Random Forest
    r2_rf = r2_score(test_df['rain'], test_df['radar_rainfall_rf'])
    
    return {
        'fold': fold,
        'mfb_train': mfb_train,
        'mfb_test': mfb_test,
        'mfb_test_rf': mfb_test_rf,
        'metrics_original': metrics_original,
        'metrics_rf': metrics_rf,
        'r2_score_rf': r2_rf,  # Add R2 score to the results
        'test_data': test_df,
        'best_model': best_model
    }

# Function to print results
def print_results(results):
    methods = ['original', 'rf']  # Changed to match the keys in the results dictionary
    print("\nAverage Metrics Across Folds:")
    for method in methods:
        print(f"\n{method.capitalize()} Method:")
        for metric in results[0][f'metrics_{method}'].keys():
            avg_metric = np.mean([r[f'metrics_{method}'][metric] for r in results])
            print(f"{metric}: {avg_metric:.4f}")
    
    print(f"\nAverage MFB (Train): {np.mean([r['mfb_train'] for r in results]):.4f}")
    print(f"Average MFB (Test Original): {np.mean([r['mfb_test'] for r in results]):.4f}")
    print(f"Average MFB (Test Random Forest): {np.mean([r['mfb_test_rf'] for r in results]):.4f}")

if __name__ == "__main__":
    # Paths
    #radar_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    #train_dir = './train_test_validate/'
    #test_dir = './train_test_validate/'
    model_save_path = '../3output/0models/rf_model_best.joblib'

    # Ensure the model save directory exists
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

    # Calculate total radar rainfall
    logging.info("Calculating total radar rainfall...")
    #total_radar_rainfall, radar_transform = calculate_total_radar_rainfall(radar_dir)

    # Process all 20 folds
    results = []
    for fold in range(1, 21):
        result = process_fold(fold)
        results.append(result)

    # Find the best model based on R2 score
    best_result = max(results, key=lambda x: x['r2_score_rf'])
    best_model = best_result['best_model']
    best_fold = best_result['fold']

    logging.info(f"Best model found in fold {best_fold} with R2 score: {best_result['r2_score_rf']}")

    # Save the best model
    joblib.dump(best_model, model_save_path)
    logging.info(f"Best model saved to {model_save_path}")

    logging.info("Model training and saving completed.")

    # Print results
    print_results(results)

2024-08-22 19:42:42,426 - INFO - Calculating total radar rainfall...
2024-08-22 19:42:42,428 - INFO - Processing fold 1


Fitting 20 folds for each of 108 candidates, totalling 2160 fits


2024-08-22 19:43:09,239 - INFO - Best parameters found: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
2024-08-22 19:43:09,240 - INFO - Best cross-validation score: -0.01774899271025545
2024-08-22 19:43:09,262 - INFO - Feature Importance:
2024-08-22 19:43:09,262 - INFO -           feature  importance
2        latitude    0.417423
1       longitude    0.366364
0  radar_rainfall    0.216213
2024-08-22 19:43:09,266 - INFO - Processing fold 2


Fitting 20 folds for each of 108 candidates, totalling 2160 fits


2024-08-22 19:43:39,419 - INFO - Best parameters found: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}
2024-08-22 19:43:39,420 - INFO - Best cross-validation score: 0.032447308728131265
2024-08-22 19:43:39,432 - INFO - Feature Importance:
2024-08-22 19:43:39,433 - INFO -           feature  importance
1       longitude    0.446262
2        latitude    0.376747
0  radar_rainfall    0.176991
2024-08-22 19:43:39,440 - INFO - Processing fold 3


...

Fitting 20 folds for each of 108 candidates, totalling 2160 fits


2024-08-22 19:52:15,122 - INFO - Best parameters found: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
2024-08-22 19:52:15,122 - INFO - Best cross-validation score: -0.258232104573153
2024-08-22 19:52:15,130 - INFO - Feature Importance:
2024-08-22 19:52:15,130 - INFO -           feature  importance
1       longitude    0.465091
2        latitude    0.350440
0  radar_rainfall    0.184470
2024-08-22 19:52:15,134 - INFO - Processing fold 20


Fitting 20 folds for each of 108 candidates, totalling 2160 fits


2024-08-22 19:52:45,505 - INFO - Best parameters found: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
2024-08-22 19:52:45,505 - INFO - Best cross-validation score: -0.19747097590643256
2024-08-22 19:52:45,528 - INFO - Feature Importance:
2024-08-22 19:52:45,529 - INFO -           feature  importance
2        latitude    0.431029
1       longitude    0.370082
0  radar_rainfall    0.198889
2024-08-22 19:52:45,532 - INFO - Best model found in fold 11 with R2 score: 0.5664314700747728
2024-08-22 19:52:45,579 - INFO - Best model saved to ../3output/0models/rf_model_best.joblib
2024-08-22 19:52:45,580 - INFO - Model training and saving completed.



Average Metrics Across Folds:

Original Method:
MBE: -14.3464
MAE: 36.7990
RMSE: 52.1148
PBIAS: -28.4801
NSE: -0.3836
R: 0.2496

Rf Method:
MBE: -0.6356
MAE: 24.1327
RMSE: 35.5751
PBIAS: -0.6833
NSE: 0.3530
R: 0.6294

Average MFB (Train): 1.4102
Average MFB (Test Original): 1.4323
Average MFB (Test Random Forest): 1.0162


In [2]:
'''
# ต้นแบบ แต่ใช้ file split train/test ไฟล์เดียว
# สร้างแบบจำลอง RF แล้วเซฟเพื่อนำมาใช้ปรับแก้ สมบูรณ์
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import rasterio
import logging
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import joblib

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_total_radar_rainfall(radar_dir):
    radar_files = glob(os.path.join(radar_dir, '*.tif'))
    total_rainfall = None
    transform = None
    
    for file in tqdm(radar_files, desc="Processing radar files"):
        with rasterio.open(file) as src:
            radar_data = src.read(1)  # Read the first band
            if total_rainfall is None:
                total_rainfall = radar_data
                transform = src.transform
            else:
                total_rainfall += radar_data
    
    return total_rainfall, transform

def extract_radar_rainfall(lon, lat, total_rainfall, transform, window_size=3):
    row, col = rasterio.transform.rowcol(transform, lon, lat)
    half_window = window_size // 2
    window = total_rainfall[max(0, row-half_window):row+half_window+1, 
                            max(0, col-half_window):col+half_window+1]
    return np.nanmean(window)

def optimize_random_forest(train_df, test_df, model_save_path):
    # Prepare features
    features = ['radar_rainfall', 'longitude', 'latitude']
    X_train = train_df[features]
    y_train = train_df['sum_rain_event']
    X_test = test_df[features]
    
    # Define parameter grid
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Create a base model
    rf = RandomForestRegressor(random_state=42)
    
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                               cv=20, n_jobs=-1, verbose=2, scoring='r2') # cv=20
    
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    
    # Print the best parameters
    logging.info(f"Best parameters found: {grid_search.best_params_}")
    logging.info(f"Best cross-validation score: {grid_search.best_score_}")
    
    # Use the best model to predict
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Feature importance
    feature_importance = pd.DataFrame({'feature': features, 'importance': best_model.feature_importances_})
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    logging.info("Feature Importance:")
    logging.info(feature_importance)
    
    # Save the model
    joblib.dump(best_model, model_save_path)
    logging.info(f"Model saved to {model_save_path}")
    
    return y_pred, best_model

if __name__ == "__main__":
    # Paths
    radar_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    train_data_path = '../1data/3Gauges/cross_val_sum_total_event_rain/train_split_kfolds/train_split_1.csv'
    test_data_path = '../1data/3Gauges/cross_val_sum_total_event_rain/train_split_kfolds/test_split_1.csv'
    model_save_path = '../4output/0models/rf_model_best.joblib'

    # Ensure the model save directory exists
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

    # Calculate total radar rainfall
    logging.info("Calculating total radar rainfall...")
    total_radar_rainfall, radar_transform = calculate_total_radar_rainfall(radar_dir)

    # Load train and test data
    logging.info("Loading train and test data...")
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # Extract radar rainfall for each station
    logging.info("Extracting radar rainfall for each station...")
    train_df['radar_rainfall'] = train_df.apply(lambda row: extract_radar_rainfall(row['longitude'], row['latitude'], total_radar_rainfall, radar_transform), axis=1)
    test_df['radar_rainfall'] = test_df.apply(lambda row: extract_radar_rainfall(row['longitude'], row['latitude'], total_radar_rainfall, radar_transform), axis=1)

    # Train and save the model
    logging.info("Training and saving the Random Forest model...")
    _, _ = optimize_random_forest(train_df, test_df, model_save_path)

    logging.info("Model training and saving completed.")
'''

'\n# ต้นแบบ แต่ใช้ file split train/test ไฟล์เดียว\n# สร้างแบบจำลอง RF แล้วเซฟเพื่อนำมาใช้ปรับแก้ สมบูรณ์\nimport pandas as pd\nimport numpy as np\nimport os\nfrom glob import glob\nfrom tqdm import tqdm\nimport rasterio\nimport logging\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.ensemble import RandomForestRegressor\nimport joblib\n\n# Set up logging\nlogging.basicConfig(level=logging.INFO, format=\'%(asctime)s - %(levelname)s - %(message)s\')\n\ndef calculate_total_radar_rainfall(radar_dir):\n    radar_files = glob(os.path.join(radar_dir, \'*.tif\'))\n    total_rainfall = None\n    transform = None\n    \n    for file in tqdm(radar_files, desc="Processing radar files"):\n        with rasterio.open(file) as src:\n            radar_data = src.read(1)  # Read the first band\n            if total_rainfall is None:\n                total_rainfall = radar_data\n                transform = src.transform\n            else:\n                total_rainfall += radar_data\n  

## โค้ดเรียกใช้แบบจำลอง RF เพื่อปรับแก้ฝนเรดาร์รายชั่วโมง รันปรับแก้ทั้งโฟลเดอร์ perfect!!!

In [ ]:
import numpy as np
import pandas as pd
import rasterio
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
from rasterio.plot import show
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, LogNorm, to_rgba
import matplotlib.colors as mcolors
from pyproj import CRS

def create_output_dirs(base_dir):
    dirs = ['corrected_tiffs', 'comparison_plots']
    for d in dirs:
        os.makedirs(os.path.join(base_dir, d), exist_ok=True)
    return {d: os.path.join(base_dir, d) for d in dirs}

def process_file(input_file, model, scaler, imputer, output_dirs, basin_shape, fit_imputer=False):
    print(f"\nProcessing file: {input_file}")
    with rasterio.open(input_file) as src:
        data = src.read(1)
        meta = src.meta.copy()
        nodata = src.nodata
        transform = src.transform
        
        if src.crs is None:
            print(f"Warning: CRS not defined for {input_file}. Using WGS84 as default.")
            crs = CRS.from_epsg(4326)  # WGS84
        else:
            crs = CRS.from_wkt(src.crs.to_wkt())
        
        bounds = src.bounds
        height, width = data.shape
        rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
        lons, lats = rasterio.transform.xy(transform, rows, cols)
    
    # Create a mask for valid data (inside radar coverage)
    valid_mask = (data != nodata)
    
    # Apply the mask to the original data
    data = np.where(valid_mask, data, np.nan)
    
    # Prepare features
    lons, lats = np.array(lons), np.array(lats)
    features = np.column_stack((data.ravel(), lons.ravel(), lats.ravel()))
    feature_names = ['rainfall', 'longitude', 'latitude']
    
    print(f"Features shape: {features.shape}")
    print(f"NaN values before imputation: {np.isnan(features).sum()}")
    
    if fit_imputer:
        imputer.fit(features[np.logical_and(valid_mask.ravel(), ~np.isnan(features[:,0]), features[:,0] > 0)])
    
    # Only impute valid data that is not zero or NaN
    features_imputed = features.copy()
    non_zero_mask = np.logical_and(valid_mask.ravel(), ~np.isnan(features[:,0]), features[:,0] > 0)
    features_imputed[non_zero_mask] = imputer.transform(features[non_zero_mask])
    print(f"NaN values after imputation: {np.isnan(features_imputed).sum()}")
    
    scaled_features = np.zeros_like(features_imputed)
    scaled_features[non_zero_mask] = scaler.transform(features_imputed[non_zero_mask])
    
    # Ensure feature names match those used in training
    X_pred = pd.DataFrame(scaled_features, columns=feature_names)
    corrected_values = model.predict(X_pred).reshape(data.shape)
    
    # Apply the corrections only to non-zero and non-NaN values within the radar coverage
    corrected_data = np.where(np.logical_and(valid_mask, data > 0), np.maximum(corrected_values, 0), data)
    corrected_data = np.where(valid_mask, corrected_data, np.nan)
    
    print(f"Original data - Mean: {np.nanmean(data):.2f}, Max: {np.nanmax(data):.2f}")
    print(f"Corrected data - Mean: {np.nanmean(corrected_data):.2f}, Max: {np.nanmax(corrected_data):.2f}")
    
    # Save corrected data
    output_file = os.path.join(output_dirs['corrected_tiffs'], f"corrected_{os.path.basename(input_file)}")
    meta.update(dtype=rasterio.float32, nodata=np.nan)
    with rasterio.open(output_file, 'w', **meta) as dst:
        dst.write(corrected_data.astype(rasterio.float32), 1)
    
    # Calculate difference, maintaining NaN values outside radar coverage
    difference = corrected_data - data
    
    # Create custom colormap for rainfall
    colors = ['#FFFFFF', '#D2D2FF', '#A6A6FF', '#8080FF', '#4F4FFF', '#2626FF', '#0000FF', '#00AAFF', '#00FFFF', '#00FF00', '#7BFF7B', '#A6FFA6', '#CAFFCA', '#FFFF00', '#FFA600', '#FF0000', '#C80000']
    n_bins = len(colors)
    cmap_rain = LinearSegmentedColormap.from_list('custom_rain', colors, N=n_bins)
    
    # Modify colormap to include light grey for zero values
    cmap_rain_with_grey = cmap_rain.copy()
    cmap_rain_with_grey.set_under('#dadada')  # Set color for values below vmin
    
    # Create a softer colormap for difference
    soft_colors = ['#4575B4', '#91BFDB', '#E0F3F8', '#FFFFBF', '#FEE090', '#FC8D59', '#D73027']
    cmap_diff = LinearSegmentedColormap.from_list('soft_diff', soft_colors, N=256)
    cmap_diff.set_bad('white')  # Set color for NaN values in difference plot
    
    # Create comparison plot
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
    
    for ax, data_to_plot, title, cmap in zip([ax1, ax2, ax3], 
                                             [data, corrected_data, difference], 
                                             ['Original Radar Data', 'Corrected Radar Data', 'Difference (Corrected - Original)'],
                                             [cmap_rain_with_grey, cmap_rain_with_grey, cmap_diff]):
        # Flip the data vertically for correct orientation
        if cmap == cmap_diff:
            # For difference plot, use symmetric vmin/vmax
            vmin_diff, vmax_diff = -max(abs(np.nanmin(difference)), abs(np.nanmax(difference))), max(abs(np.nanmin(difference)), abs(np.nanmax(difference)))
            masked_diff = np.ma.masked_where(np.logical_or(np.abs(data_to_plot) < 1e-6, np.isnan(data_to_plot)), data_to_plot)
            im = ax.imshow(np.flipud(masked_diff), cmap=cmap, extent=[bounds.left, bounds.right, bounds.top, bounds.bottom], vmin=vmin_diff, vmax=vmax_diff)
        else:
            # For rainfall data, use logarithmic scale with a small offset to handle zeros
            vmin_plot, vmax_plot = 0.1, np.nanmax(data_to_plot)
            norm = LogNorm(vmin=vmin_plot, vmax=vmax_plot)
            masked_data = np.ma.masked_where(np.logical_or(data_to_plot == 0, np.isnan(data_to_plot)), data_to_plot)
            im = ax.imshow(np.flipud(masked_data), cmap=cmap, norm=norm, extent=[bounds.left, bounds.right, bounds.top, bounds.bottom])
            # Plot zero values in light grey
            ax.imshow(np.flipud(np.where(data_to_plot == 0, 1, np.nan)), cmap=ListedColormap(['#dadada']), extent=[bounds.left, bounds.right, bounds.top, bounds.bottom])
        
        ax.set_title(title)
        
        basin_shape_reprojected = basin_shape.to_crs(crs)
        basin_shape_reprojected.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        plt.colorbar(im, ax=ax, label='Rainfall (mm/hr)' if cmap != cmap_diff else 'Difference (mm/hr)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dirs['comparison_plots'], f"comparison_{os.path.splitext(os.path.basename(input_file))[0]}.png"), dpi=300)
    plt.close()

    return np.nanmean(data), np.nanmax(data), np.nanmean(corrected_data), np.nanmax(corrected_data)

def main():
    print("Script started")
    
    # Paths
    input_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    output_base_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic_corrected'
    model_path = '../3output/0models/rf_model_best.joblib'
    scaler_path = '../3output/0models/scaler_new.joblib'
    
    output_dirs = create_output_dirs(output_base_dir)
    
    print("Loading model and scaler")
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("Model and scaler loaded successfully")
    print(f"Model n_features_in_: {model.n_features_in_}")
    print(f"Scaler n_features_in_: {scaler.n_features_in_}")
    
    # Print feature importances
    feature_names = ['rainfall', 'longitude', 'latitude']
    importances = model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Importance of {name}: {importance}")
    
    # Create imputers with different strategies
    imputers = {
        'mean': SimpleImputer(strategy='mean'),
        'median': SimpleImputer(strategy='median'),
        'constant': SimpleImputer(strategy='constant', fill_value=0)
    }
    
    # Load Chao Phraya Basin shapefile
    basin_shape = gpd.read_file('../1data/1GIS/0base_map_gis/Chaophraya_Basins_others_ONWR_Law_WGS84Geo.shp')
    
    input_files = [f for f in os.listdir(input_hourly_dir) if f.endswith('.tif')]
    print(f"Found {len(input_files)} .tif files")
    
    if not input_files:
        print("No .tif files found in the input directory")
        return
    
    # Find global min and max for consistent color scaling
    global_min = float('inf')
    global_max = float('-inf')
    for file in input_files[:10]:  # Adjust if processing more files
        input_file = os.path.join(input_hourly_dir, file)
        with rasterio.open(input_file) as src:
            data = src.read(1)
            valid_data = data[data != src.nodata]
            global_min = min(global_min, np.nanmin(valid_data))
            global_max = max(global_max, np.nanmax(valid_data))

    # Adjust color range slightly to ensure all data is visible
    vmin, vmax = global_min * 0.9, global_max * 1.1
    
    results = {strategy: [] for strategy in imputers.keys()}
    
    for strategy, imputer in imputers.items():
        print(f"\nProcessing files with {strategy} imputation strategy")
        for i, file in enumerate(tqdm(input_files)):  # Process all files
        #for i, file in enumerate(tqdm(input_files[:10])):  # Process first 10 files
            input_file = os.path.join(input_hourly_dir, file)
            orig_mean, orig_max, corr_mean, corr_max = process_file(
                input_file, model, scaler, imputer, output_dirs, basin_shape, fit_imputer=(i==0)
            )
            results[strategy].append((orig_mean, orig_max, corr_mean, corr_max))
    
    # Plot results
    strategies = list(imputers.keys())
    x = np.arange(len(results[strategies[0]]))
    width = 0.25
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 20))
    
    for i, strategy in enumerate(strategies):
        means = [r[2] for r in results[strategy]]
        ax1.bar(x + i*width, means, width, label=f'{strategy} imputation')
    
    ax1.set_ylabel('Corrected Mean Rainfall')
    ax1.set_title('Comparison of Imputation Strategies - Mean Rainfall')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels([f'File {i+1}' for i in range(len(x))])
    ax1.legend()
    
    for i, strategy in enumerate(strategies):
        maxes = [r[3] for r in results[strategy]]
        ax2.bar(x + i*width, maxes, width, label=f'{strategy} imputation')
    
    ax2.set_ylabel('Corrected Max Rainfall')
    ax2.set_title('Comparison of Imputation Strategies - Max Rainfall')
    ax2.set_xticks(x + width)
    ax2.set_xticklabels([f'File {i+1}' for i in range(len(x))])
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_base_dir, 'imputation_strategy_comparison.png'))
    plt.close()
    
    print("Script completed")

if __name__ == "__main__":
    main()

Script started
Loading model and scaler
Model and scaler loaded successfully
Model n_features_in_: 3
Scaler n_features_in_: 3
Importance of rainfall: 0.20793344664295807
Importance of longitude: 0.32822411945752383
Importance of latitude: 0.46384243389951796
Found 235 .tif files

Processing files with mean imputation strategy


  0%|                                                                                          | 0/235 [00:00<?, ?it/s]


Processing file: D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic\2018071507_2000m.tif
Features shape: (250000, 3)
NaN values before imputation: 114311
NaN values after imputation: 114311


C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- rainfall
Feature names seen at fit time, yet now missing:
- radar_rainfall

  warnings.warn(message, FutureWarning)


Original data - Mean: 0.01, Max: 4.38
Corrected data - Mean: 0.19, Max: 17.07


...


Processing file: D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic\2018072122_2000m.tif
Features shape: (250000, 3)
NaN values before imputation: 114278
NaN values after imputation: 114278
Original data - Mean: 0.06, Max: 19.93
Corrected data - Mean: 0.61, Max: 49.43


 66%|████████████████████████████████████████████████████▊                           | 155/235 [12:05<06:05,  4.57s/it]C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- rainfall
Feature names seen at fit time, yet now missing:
- radar_rainfall

  warnings.warn(message, FutureWarning)



Processing file: D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic\2018072123_2000m.tif
Features shape: (250000, 3)
NaN values before imputation: 114348
NaN values after imputation: 114348
Original data - Mean: 0.03, Max: 10.41
Corrected data - Mean: 0.51, Max: 35.83


 66%|█████████████████████████████████████████████████████                           | 156/235 [12:10<05:58,  4.54s/it]C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- rainfall
Feature names seen at fit time, yet now missing:
- radar_rainfall

  warnings.warn(message, FutureWarning)



Processing file: D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic\2018072200_2000m.tif
Features shape: (250000, 3)
NaN values before imputation: 114303
NaN values after imputation: 114303
Original data - Mean: 0.03, Max: 15.10
Corrected data - Mean: 0.42, Max: 49.43


In [ ]:
'''    
# เหมือนจะได้แล้ว แต่ค่า mean ดันสูง
import numpy as np
import rasterio
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import geopandas as gpd

def process_file(input_file, model, scaler, imputer, fit_imputer=False):
    print(f"Processing file: {input_file}")
    with rasterio.open(input_file) as src:
        data = src.read(1)
        height, width = data.shape
        rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
        lons, lats = rasterio.transform.xy(src.transform, rows, cols)
    
    lons = np.array(lons)
    lats = np.array(lats)
    
    # Create features array with radar data, longitude, and latitude
    features = np.column_stack((data.ravel(), lons.ravel(), lats.ravel()))
    
    print(f"Features shape: {features.shape}")
    print(f"NaN values before imputation: {np.isnan(features).sum()}")
    
    # Fit the imputer if this is the first file
    if fit_imputer:
        imputer.fit(features)
    
    # Impute NaN values
    features_imputed = imputer.transform(features)
    print(f"NaN values after imputation: {np.isnan(features_imputed).sum()}")
    
    scaled_features = scaler.transform(features_imputed)
    corrected_values = model.predict(scaled_features).reshape(data.shape)
    corrected_data = np.maximum(corrected_values, 0)
    
    print(f"Original data - Mean: {np.nanmean(data):.2f}, Max: {np.nanmax(data):.2f}")
    print(f"Corrected data - Mean: {np.mean(corrected_data):.2f}, Max: {np.max(corrected_data):.2f}")

def main():
    print("Script started")
    
    # Paths
    input_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    model_path = '../4output/0models/rf_model_best.joblib'
    scaler_path = '../4output/0models/scaler_new.joblib'
    
    print("Loading model and scaler")
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("Model and scaler loaded successfully")
    print(f"Model n_features_in_: {model.n_features_in_}")
    print(f"Scaler n_features_in_: {scaler.n_features_in_}")
    
    # Create an imputer
    imputer = SimpleImputer(strategy='mean')
    
    print("Getting list of input files")
    input_files = [f for f in os.listdir(input_hourly_dir) if f.endswith('.tif')]
    print(f"Found {len(input_files)} .tif files")
    
    if not input_files:
        print("No .tif files found in the input directory")
        return
    
    print("Processing files")
    for i, file in enumerate(input_files[:5]):  # Process only the first 5 files for testing
        input_file = os.path.join(input_hourly_dir, file)
        process_file(input_file, model, scaler, imputer, fit_imputer=(i==0))
    
    print("Script completed")

if __name__ == "__main__":
    main()
    
'''    

In [ ]:
'''
# ต้นแบบ ดีมาก
# โค้ดเรียกใช้แบบจำลอง RF เพื่อปรับแก้ฝนเรดาร์รายชั่วโมง รันปรับแก้ทั้งโฟลเดอร์ perfect!!!
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import logging
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Keep the existing functions (extract_features, create_and_save_scaler, correct_single_radar_file, print_statistics) as they are

def visualize_correction(original_data, corrected_data, output_plot, input_file, basin_shape):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
    
    with rasterio.open(input_file) as src:
        extent = [src.bounds.left, src.bounds.right, src.bounds.top, src.bounds.bottom]  # Note the swap of top and bottom
    
    # Flip the data vertically
    original_data = np.flipud(original_data)
    corrected_data = np.flipud(corrected_data)
    
    # Calculate difference
    difference = corrected_data - original_data
    
    # Plot original data
    basin_shape.plot(ax=ax1, color='none', edgecolor='black', linewidth=0.5)
    im1 = ax1.imshow(original_data, cmap='viridis', extent=extent, vmin=0, vmax=np.nanpercentile(original_data, 99))
    ax1.set_title('Original Radar Data')
    plt.colorbar(im1, ax=ax1, label='Rainfall (mm)')
    
    # Plot corrected data
    basin_shape.plot(ax=ax2, color='none', edgecolor='black', linewidth=0.5)
    im2 = ax2.imshow(corrected_data, cmap='viridis', extent=extent, vmin=0, vmax=np.nanpercentile(corrected_data, 99))
    ax2.set_title('Corrected Radar Data')
    plt.colorbar(im2, ax=ax2, label='Rainfall (mm)')
    
    # Plot difference
    basin_shape.plot(ax=ax3, color='none', edgecolor='black', linewidth=0.5)
    vmax = np.nanpercentile(np.abs(difference), 99)
    im3 = ax3.imshow(difference, cmap='RdBu_r', extent=extent, vmin=-vmax, vmax=vmax)
    ax3.set_title('Difference (Corrected - Original)')
    plt.colorbar(im3, ax=ax3, label='Rainfall Difference (mm)')
    
    for ax in (ax1, ax2, ax3):
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
    
    plt.tight_layout()
    plt.savefig(output_plot, dpi=300, bbox_inches='tight')
    plt.close()

    logging.info(f"Visualization saved to {output_plot}")

def process_all_files(input_dir, output_dir, model, scaler, basin_shape):
    input_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]
    
    if not input_files:
        logging.error(f"No .tif files found in {input_dir}")
        return
    
    for file in tqdm(input_files, desc="Processing files"):
        input_file = os.path.join(input_dir, file)
        output_file = os.path.join(output_dir, f"corrected_{file}")
        output_plot = os.path.join(output_dir, f"comparison_{os.path.splitext(file)[0]}.png")
        
        logging.info(f"Processing file: {input_file}")
        
        # Correct the file
        original_data, corrected_data, meta = correct_single_radar_file(input_file, output_file, model, scaler)
        
        # Visualize the results
        visualize_correction(original_data, corrected_data, output_plot, input_file, basin_shape)
        
        # Print statistics
        print_statistics(original_data, corrected_data)
        
        logging.info(f"Corrected file saved to: {output_file}")

if __name__ == "__main__":
    # Paths
    input_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    output_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic_corrected'
    model_path = '../4output/0models/rf_model_best.joblib'
    scaler_path = '../4output/0models/scaler_new.joblib'
    
    # Load Chao Phraya Basin shapefile
    basin_shape = gpd.read_file('../1data/1GIS/0base_map_gis/Chaophraya_Basins_others_ONWR_Law_WGS84Geo.shp')
    
    # Create output directory if it doesn't exist
    os.makedirs(output_hourly_dir, exist_ok=True)
    os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
    
    # Load the saved model
    logging.info(f"Loading model from {model_path}")
    model = joblib.load(model_path)
    
    # Load or create scaler
    if os.path.exists(scaler_path):
        logging.info(f"Loading scaler from {scaler_path}")
        scaler = joblib.load(scaler_path)
    else:
        logging.info(f"Creating new scaler.")
        input_files = [f for f in os.listdir(input_hourly_dir) if f.endswith('.tif')]
        if not input_files:
            logging.error(f"No .tif files found in {input_hourly_dir}")
            exit(1)
        sample_file = os.path.join(input_hourly_dir, input_files[0])
        scaler = create_and_save_scaler(sample_file, scaler_path)
    
    # Process all files
    process_all_files(input_hourly_dir, output_hourly_dir, model, scaler, basin_shape)
    
    logging.info("All files processed. Please check the output directory for results.")
'''    

In [ ]:
'''
# โค้ดเรียกใช้แบบจำลอง RF เพื่อปรับแก้ฝนเรดาร์รายชั่วโมง แบบระบุวันเวลาได้ perfect!!!
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import logging
from datetime import datetime
import geopandas as gpd
from rasterio.mask import mask

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_features(data, lons, lats):
    features = np.stack([data, lons, lats], axis=-1)
    return features

def create_and_save_scaler(input_file, scaler_path):
    with rasterio.open(input_file) as src:
        radar_data = src.read(1)
        height, width = radar_data.shape
        rows, cols = np.mgrid[0:height, 0:width]
        lons, lats = rasterio.transform.xy(src.transform, rows, cols)
    
    X = extract_features(radar_data, lons, lats)
    X_flat = X.reshape(-1, X.shape[-1])
    scaler = StandardScaler()
    scaler.fit(X_flat)
    joblib.dump(scaler, scaler_path)
    logging.info(f"Scaler created and saved to {scaler_path}")
    return scaler

def correct_single_radar_file(input_file, output_file, model, scaler):
    with rasterio.open(input_file) as src:
        radar_data = src.read(1)
        
        # Create a mask for areas inside radar coverage (non-NaN in original data)
        inside_radar_mask = ~np.isnan(radar_data)
        
        # Create a mask for areas with no rain (0 mm) inside radar coverage
        no_rain_mask = (radar_data == 0) & inside_radar_mask
        
        # Create a grid of longitude and latitude values
        height, width = radar_data.shape
        rows, cols = np.mgrid[0:height, 0:width]
        lons, lats = rasterio.transform.xy(src.transform, rows, cols)
        
        # Extract features
        X = extract_features(radar_data, lons, lats)
        
        # Handle NaNs before scaling (only for areas inside radar coverage)
        for i in range(X.shape[-1]):
            feature = X[..., i]
            feature[inside_radar_mask] = np.nan_to_num(feature[inside_radar_mask], nan=np.nanmean(feature[inside_radar_mask]))
        
        # Reshape for scaling
        X_flat = X.reshape(-1, X.shape[-1])
        
        # Scale features
        X_scaled = scaler.transform(X_flat)
        
        # Apply the correction only to areas inside radar coverage
        corrected_data = np.full_like(radar_data, np.nan)
        corrected_data[inside_radar_mask] = model.predict(X_scaled[inside_radar_mask.ravel()]).reshape(inside_radar_mask.sum())
        
        # Ensure corrected values are non-negative
        corrected_data = np.maximum(corrected_data, 0)
        
        # Apply the no-rain mask to set those areas to zero
        corrected_data[no_rain_mask] = 0
        
        # Write the corrected data to a new GeoTIFF file
        with rasterio.open(output_file, 'w', **src.meta) as dst:
            dst.write(corrected_data, 1)
    
    return radar_data, corrected_data, src.meta

def visualize_correction(original_data, corrected_data, output_plot, input_file, basin_shape):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
    
    with rasterio.open(input_file) as src:
        extent = [src.bounds.left, src.bounds.right, src.bounds.top, src.bounds.bottom]  # Note the swap of top and bottom
    
    # Flip the data vertically
    original_data = np.flipud(original_data)
    corrected_data = np.flipud(corrected_data)
    
    # Calculate difference
    difference = corrected_data - original_data
    
    # Plot original data
    basin_shape.plot(ax=ax1, color='none', edgecolor='red', linewidth=0.5)
    im1 = ax1.imshow(original_data, cmap='viridis', extent=extent, vmin=0, vmax=np.nanpercentile(original_data, 99))
    ax1.set_title('Original Radar Data')
    plt.colorbar(im1, ax=ax1, label='Rainfall (mm)')
    
    # Plot corrected data
    basin_shape.plot(ax=ax2, color='none', edgecolor='red', linewidth=0.5)
    im2 = ax2.imshow(corrected_data, cmap='viridis', extent=extent, vmin=0, vmax=np.nanpercentile(corrected_data, 99))
    ax2.set_title('Corrected Radar Data')
    plt.colorbar(im2, ax=ax2, label='Rainfall (mm)')
    
    # Plot difference
    basin_shape.plot(ax=ax3, color='none', edgecolor='black', linewidth=0.5)
    vmax = np.nanpercentile(np.abs(difference), 99)
    im3 = ax3.imshow(difference, cmap='RdBu_r', extent=extent, vmin=-vmax, vmax=vmax)
    ax3.set_title('Difference (Corrected - Original)')
    plt.colorbar(im3, ax=ax3, label='Rainfall Difference (mm)')
    
    for ax in (ax1, ax2, ax3):
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
    
    plt.tight_layout()
    plt.savefig(output_plot, dpi=300, bbox_inches='tight')
    plt.close()

    logging.info(f"Visualization saved to {output_plot}")

def print_statistics(original_data, corrected_data):
    # Mask for areas inside radar coverage
    inside_radar_mask = ~np.isnan(original_data)
    
    original_stats = {
        'min': np.nanmin(original_data),
        'max': np.nanmax(original_data),
        'mean': np.nanmean(original_data),
        'std': np.nanstd(original_data)
    }
    corrected_stats = {
        'min': np.nanmin(corrected_data[inside_radar_mask]),
        'max': np.nanmax(corrected_data[inside_radar_mask]),
        'mean': np.nanmean(corrected_data[inside_radar_mask]),
        'std': np.nanstd(corrected_data[inside_radar_mask])
    }
    
    logging.info("Original Data Statistics (inside radar coverage):")
    for key, value in original_stats.items():
        logging.info(f"{key}: {value:.4f}")
    
    logging.info("\nCorrected Data Statistics (inside radar coverage):")
    for key, value in corrected_stats.items():
        logging.info(f"{key}: {value:.4f}")

if __name__ == "__main__":
    # Paths
    input_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic'
    output_hourly_dir = r'D:\1Yang\0Research\0_2566_radar_mosaic\00mos_chaophraya\00run_batch_acchr_codes\2output\0Hourly\0Sontihn_RF\0CBB_Pulse\0output_mosaic_corrected'
    model_path = '../4output/0models/rf_model_best.joblib'
    scaler_path = '../4output/0models/scaler_new.joblib'
    
    # Load Chao Phraya Basin shapefile
    basin_shape = gpd.read_file('../1data/1GIS/0base_map_gis/Chaophraya_Basins_others_ONWR_Law_WGS84Geo.shp')
    
    # Create output directory if it doesn't exist
    os.makedirs(output_hourly_dir, exist_ok=True)
    os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
    
    # Load the saved model
    logging.info(f"Loading model from {model_path}")
    model = joblib.load(model_path)
    
    # Load or create scaler
    if os.path.exists(scaler_path):
        logging.info(f"Loading scaler from {scaler_path}")
        scaler = joblib.load(scaler_path)
    else:
        logging.info(f"Creating new scaler.")
        input_files = [f for f in os.listdir(input_hourly_dir) if f.endswith('.tif')]
        if not input_files:
            logging.error(f"No .tif files found in {input_hourly_dir}")
            exit(1)
        sample_file = os.path.join(input_hourly_dir, input_files[0])
        scaler = create_and_save_scaler(sample_file, scaler_path)
    
    # Get user input for date and time
    date_input = input("Enter the date and time in format YYYYMMDDHH (e.g., 2018071719): ")
    target_datetime = datetime.strptime(date_input, "%Y%m%d%H")
    
    # Find the matching file
    target_file = None
    for file in os.listdir(input_hourly_dir):
        if file.startswith(target_datetime.strftime("%Y%m%d%H")) and file.endswith('.tif'):
            target_file = file
            break
    
    if target_file is None:
        logging.error(f"No file found for the specified date and time: {date_input}")
        exit(1)
    
    input_file = os.path.join(input_hourly_dir, target_file)
    output_file = os.path.join(output_hourly_dir, f"corrected_{target_file}")
    output_plot = os.path.join(output_hourly_dir, f"comparison_{os.path.splitext(target_file)[0]}.png")
    
    logging.info(f"Processing file: {input_file}")
    
    # Correct the single file
    original_data, corrected_data, meta = correct_single_radar_file(input_file, output_file, model, scaler)
    
    # Visualize the results
    visualize_correction(original_data, corrected_data, output_plot, input_file, basin_shape)
    
    # Print statistics
    print_statistics(original_data, corrected_data)
    
    logging.info(f"Corrected file saved to: {output_file}")
    logging.info("Process completed. Please check the output directory for results.")
'''    

## Next
* นำผลทั้งหมดมาเรียงในสไลด์ พร้อมโค้ดที่ใช้สร้าง
* สร้างโครงในเปเปอร์
* เขียนอธิบายผล 
    * เป็นข้อ ๆ อภิปรายบางส่วน เขียนเน้นเรื่องการพัฒนาโค้ดโมเสค การใช้ DEM เข้ามาเป็น QI การ interpolate กลไกของโค้ด
    * เอาข้อมูล report ความครบถ้วน มาอธิบาย
    * 
* เขียนอภิปราย
    * ผลการ improve แม้ว่าจะไม่ sig
    * ผลจากแบบจำลอง RF ของฝนปรับแก้ ยกกรณีชั่วโมงตัวอย่าง
* เขียนกรอบแนวคิด วาดรูป เขียนวิธีการที่ใช้ ข้อมูลที่ใช้ 
* เขียน introduction
* เขียนสรุป ตามคำถาม และสิ่งค้นพบ
* เขียนบทคัดย่อ 
* หาเปเปอร์มาอ้างอิงเพิ่ม และอภิปราย